### This notebook looks at plots in pre-run dipy pipeplines.

Pipeline: http://localhost:8893/edit/Code/whobpyt/Intro_to_dMRI_workshop/SC_Mtx_hig_res_version_2.py



In [4]:
# importage

import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
from matplotlib import cm

import os,sys,glob,numpy as np, pandas as pd

from skimage import measure

import nibabel as nib
from nilearn.plotting import plot_surf, plot_surf_stat_map, plot_roi, plot_anat, plot_surf_roi

%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from nilearn.image import index_img

from dipy.core.gradients import gradient_table
from dipy.reconst import shm
from dipy.direction import peaks

from dipy.tracking import utils
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.io.dpy import Dpy


import seaborn as sns

from matplotlib.colors import LinearSegmentedColormap

from dipy.tracking.distances import approx_polygon_track

In [5]:
import nilearn

In [6]:
%%time
# Load the DWI data

bvecs_file = '/external/rprshnas01/public_datasets/HCP/HCP_S900/100307/T1w/Diffusion/bvecs'
bvecs_dat = np.loadtxt(bvecs_file)

bvals_file = '/external/rprshnas01/public_datasets/HCP/HCP_S900/100307/T1w/Diffusion/bvals'
bvals_dat = np.loadtxt(bvals_file)


# laoding a dwi file is similar to loading in a dconn file ...

dwi_file = '/external/rprshnas01/public_datasets/HCP/HCP_S900/100307/T1w/Diffusion/data.nii.gz'
dwi_img = nib.load(dwi_file)
dwi_dat = dwi_img.get_data()

b0_img = index_img(dwi_img,0)

# load brain mask file ...

nbm_file = '/external/rprshnas01/public_datasets/HCP/HCP_S900/100307/T1w/Diffusion/nodif_brain_mask.nii.gz'
nbm_img = nib.load(nbm_file)
nbm_dat = nbm_img.get_data()

CPU times: user 22.8 s, sys: 5.14 s, total: 27.9 s
Wall time: 28 s


In [7]:
gtab = gradient_table(bvals_dat, bvecs_dat)

affine = dwi_img.affine

In [8]:
%%time
# Take approx. 5 mins to run.

from dipy.reconst import dti
from dipy.segment.mask import median_otsu
from dipy.tracking import utils

dwi_data = dwi_img.get_fdata()

# Specify the volume index to the b0 volumes
dwi_data, dwi_mask = median_otsu(dwi_data, vol_idx=[0], numpass=1)

dti_model = dti.TensorModel(gtab)

# This step may take a while
dti_fit = dti_model.fit(dwi_data, mask=dwi_mask)

# Create the seeding mask
fa_img = dti_fit.fa
seed_mask = fa_img.copy()


# Create the seeds
seeds = utils.seeds_from_mask(seed_mask, affine=affine, density=1)

CPU times: user 2min 44s, sys: 39 s, total: 3min 23s
Wall time: 3min 26s


In [9]:
seed_mask[seed_mask >= 0.2] = 1
seed_mask[seed_mask < 0.2] = 0

In [10]:
np.unique(seed_mask)

array([0., 1.])

In [11]:
from scipy import ndimage  # To rotate image for visualization purposes
import matplotlib.pyplot as plt
from dipy.reconst.shm import CsaOdfModel
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

In [15]:
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion

In [12]:
sh_order = 2

In [13]:
%%time
csa_model = CsaOdfModel(gtab, sh_order=sh_order)

CPU times: user 2.48 ms, sys: 1.87 ms, total: 4.34 ms
Wall time: 3.08 ms


In [14]:
%%time
gfa = csa_model.fit(dwi_data, mask=seed_mask).gfa

MemoryError: Unable to allocate 3.68 GiB for an array with shape (145, 174, 145, 270) and data type float32

In [16]:
%%time
stopping_criterion = BinaryStoppingCriterion(nbm_dat)

CPU times: user 2.03 ms, sys: 9.46 ms, total: 11.5 ms
Wall time: 18.5 ms


In [17]:
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
from dipy.reconst.csdeconv import auto_response_ssst

In [18]:
%%time

# Takes approx. 2 mins to run ...

response, ratio = auto_response_ssst(gtab, dwi_data, roi_radii=10, fa_thr=0.7)
csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order=sh_order)
csd_fit = csd_model.fit(dwi_data, mask=seed_mask)

100%|██████████| 337442/337442.0 [00:47<00:00, 7061.23it/s]

CPU times: user 46 s, sys: 4.58 s, total: 50.6 s
Wall time: 52.1 s


In [20]:
from dipy.direction import ProbabilisticDirectionGetter
from dipy.data import small_sphere
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_tractogram
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines

In [ ]:
%%time

# Takes approx. 9-13 minutes to run at seed_density = 1. --> ~700,000 streamlines
# Takes approx. 90 minutes to run at seed_density = 2. --> ~5,300,000 streamlines

fod = csd_fit.odf(small_sphere)
pmf = fod.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30.,
                                                sphere=small_sphere)
streamline_generator = LocalTracking(prob_dg, stopping_criterion, seeds,
                                     affine, step_size=.5)
streamlines = Streamlines(streamline_generator)
sft = StatefulTractogram(streamlines, dwi_img, Space.RASMM)

In [ ]:
len(streamlines)

In [ ]:
%%time
# This step take 3 minutes to run. 
streamlines_ds = np.array([approx_polygon_track(s) for s in streamlines])

#### Str Conn Mtx

In [ ]:
# Load the label file and brain mask file

label_file_path = '/external/rprshnas01/netdata_kcni/jglab/MemberSpaces/Data/Shrey/PyTepFit/Data/fs_directory/100307/mri/100307_Schaefer2018_400Parcels_7Networks_rewritten.nii.gz'
mask_file_path = '/external/rprshnas01/public_datasets/HCP/HCP_S900/100307/T1w/Diffusion/nodif_brain_mask.nii.gz'

label_img = nib.load(label_file_path)
mask_img = nib.load(mask_file_path)

In [ ]:
%%time
resampled_label_dat = nilearn.image.resample_img(label_file_path, target_affine=nbm_img.affine, target_shape=mask_img.shape,interpolation='nearest')

In [ ]:
resampled_label_data = resampled_label_dat.get_data()

In [ ]:
%%time
conn_mat, grouping = utils.connectivity_matrix(streamlines_ds,
                                               b0_img.affine,
                                               resampled_label_data,
                                               return_mapping=True,
                                               mapping_as_streamlines=True)

In [ ]:
conn_mat = conn_mat[1:, 1:]

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

sns.heatmap(np.log1p(conn_mat), ax=ax, vmax=1)
ax.set_aspect('equal')

In [ ]:
from nilearn import plotting as nplot


nplot.plot_roi(resampled_label_dat, dwi_img.slicer[:, :, :, 0])
